# Download CHIRTS Data from IRI

In [22]:
%load_ext jupyter_black
import os
from pathlib import Path
import requests
from bs4 import BeautifulSoup
import geopandas as gpd
import rioxarray as rio

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [23]:
data_dir = Path(os.getenv("HAM_DIR")) / "temperature/"
mon_chirts_dir = data_dir / "monthly_chirts/"
shp_dir = Path(os.getenv("AA_DATA_DIR")) / "public/raw/ner/cod_ab/ner_cod_ab/"

In [24]:
ner_shp_adm0 = gpd.read_file(shp_dir / "adm0.shp.zip")
ner_shp_adm0.total_bounds

array([ 0.16625  , 11.69697  , 15.99564  , 23.5331862])

The next two cells take a lot of time to download as they are daily values. Looking for monthly values.

In [25]:
# URL_Tmax = "https://iridl.ldeo.columbia.edu/SOURCES/.UCSB/.CHIRTS/.v1.0/.daily/.global/.0p05/.tmax/X/%280.1E%29%2816E%29RANGEEDGES/Y/%2811.6N%29%2823.6N%29RANGEEDGES/ngridtable/4+ncoltable.html?tabopt.N=5&tabopt.1=text&tabopt.2=text&tabopt.3=text&tabopt.4=text&tabopt.5=blankNaN&NaNmarker=&tabtype=csv&eol=CR-LF+%28DOS%2FWindows%29&filename=datafile.csv"
# URL_Tmax = "https://iridl.ldeo.columbia.edu/SOURCES/.UCSB/.CHIRTS/.v1.0/.daily/.global/.0p05/.tmax/X/%280.1E%29%2816E%29RANGEEDGES/Y/%2811.6N%29%2823.6NN%29RANGEEDGES/T/%281%20Jan%201983%29%2831%20Dec%202016%29RANGEEDGES/T/%281%20Jan%201983%29%2831%20Dec%202016%29RANGEEDGES/[Y+X]datatable.tsv"
# response_tmax = requests.get(URL_Tmax)
# open(data_dir / "CHIRTSTmaxdatafile.tsv", "wb").write(response_tmax.content)

In [26]:
# URL_Tmin = "https://iridl.ldeo.columbia.edu/SOURCES/.UCSB/.CHIRTS/.v1.0/.daily/.global/.0p05/.tmin/X/%280.1E%29%2816E%29RANGEEDGES/Y/%2811.6N%29%2823.6N%29RANGEEDGES/ngridtable/4+ncoltable.html?tabopt.N=5&tabopt.1=text&tabopt.2=text&tabopt.3=text&tabopt.4=text&tabopt.5=blankNaN&NaNmarker=&tabtype=csv&eol=CR-LF+%28DOS%2FWindows%29&filename=datafile.csv"
# URL_Tmin = "https://iridl.ldeo.columbia.edu/SOURCES/.UCSB/.CHIRTS/.v1.0/.daily/.global/.0p05/.tmin/X/%280.1E%29%2816E%29RANGEEDGES/Y/%2811.6N%29%2823.6N%29RANGEEDGES/[Y+X]datatable.tsv"
# response_tmin = requests.get(URL_Tmin)
# open(data_dir / "CHIRTSTmindatafile.tsv", "wb").write(response_tmin.content)

This cell can download monthly values and rasters clipped based on bounds.

In [27]:
# Alternative Source
# This data is global
URL = "https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/fews/web/global/monthly/chirts/tmax/downloads/monthly/"


def list_files(URL, ext):
    page = requests.get(URL, verify=False).text
    soup = BeautifulSoup(page, "html.parser")
    for node in soup.find_all("a"):
        if node.get("href").endswith(ext):
            file_url = [
                URL + "/" + node.get("href")
                for node in soup.find_all("a")
                if node.get("href").endswith(ext)
            ]
            file_list = [
                node.get("href")
                for node in soup.find_all("a")
                if node.get("href").endswith(ext)
            ]
    return file_list


chirts_file_list = list_files(URL=URL, ext="tif")

c:\Users\pauni\Desktop\Work\OCHA\GitHub\VirtualEnv\hamenv\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'edcintl.cr.usgs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [28]:
# this clips the raster file to only Niger bounds
for file in chirts_file_list:
    if file in chirts_file_list:
        input_raster = rio.open_rasterio(URL + file)
        rast_clip = input_raster.rio.clip_box(*ner_shp_adm0.total_bounds)
        rast_clip.rio.to_raster(mon_chirts_dir / file)

In [ ]:
# this downloads the global dataset
#for file in chirts_file_list:
#    if file in chirts_file_list:
#        req = requests.get(URL + file, verify=False, stream=True)
#        open(data_dir / file, "wb").write(req.content)

c:\Users\pauni\Desktop\Work\OCHA\GitHub\VirtualEnv\hamenv\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'edcintl.cr.usgs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
